In [38]:
import tensorflow as tf
import numpy as np
import os 
import cv2
import random
from tqdm import tqdm
from tensorflow.keras.layers import LSTM, Bidirectional, Conv2D, Dense, Flatten, MaxPooling2D, TimeDistributed, Reshape
from tensorflow.keras.models import Sequential
import time
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from random import shuffle
from sklearn.model_selection import KFold
from tensorflow.keras import optimizers

In [4]:
LR = 0.0001
BATCH_SIZE = 64
MOMENTUM = 0.9
IMG_SIZE = 224
EPOCHS = 5

TRAIN_DIR = './Peliculas/Data/train/'
TEST_DIR = './Peliculas/Data/valid/'

In [17]:
TRAIN_DATA = []
for folder in os.listdir(TRAIN_DIR):
    for files in tqdm(os.listdir(os.path.join(TRAIN_DIR, folder))):
        path = os.path.join(os.path.join(TRAIN_DIR, folder), files)
        image = cv2.resize(cv2.imread(path), (IMG_SIZE, IMG_SIZE))
        if folder == 'fight':
            TRAIN_DATA.append([np.array(image), [1, 0]])
        elif folder == 'nofight':
            TRAIN_DATA.append([np.array(image), [0, 1]])

shuffle(TRAIN_DATA)
np.save('movie_training_data.npy', TRAIN_DATA)

100%|██████████| 4410/4410 [00:18<00:00, 243.31it/s]


In [18]:
TEST_DATA = []
for folder in os.listdir(TEST_DIR):
    for files in tqdm(os.listdir(os.path.join(TEST_DIR, folder))):
        path = os.path.join(os.path.join(TEST_DIR, folder), files)
        image = cv2.resize(cv2.imread(path), (IMG_SIZE, IMG_SIZE))
        if folder == 'fight':
            TEST_DATA.append([np.array(image), [1, 0]])
        elif folder == 'nofight':
            TEST_DATA.append([np.array(image), [0, 1]])

shuffle(TEST_DATA)
np.save('movie_testing_data.npy', TEST_DATA)

100%|██████████| 539/539 [00:01<00:00, 274.18it/s]


In [25]:
DATA_SET = []
for folders in os.listdir('./Peliculas/Data/'):
    for folder in os.listdir('./Peliculas/Data/'+str(folders)):
        folder_path = './Peliculas/Data/' + folders
        for files in tqdm(os.listdir(os.path.join(folder_path, folder))):
            path = os.path.join(os.path.join(folder_path, folder), files)
            image = cv2.resize(cv2.imread(path), (IMG_SIZE, IMG_SIZE))
            if folder == 'fight':
                DATA_SET.append([np.array(image), [1, 0]])
            elif folder == 'nofight':
                DATA_SET.append([np.array(image), [0, 1]])

shuffle(DATA_SET)
np.save('movie_data.npy', DATA_SET)

100%|██████████| 539/539 [00:02<00:00, 260.82it/s]
C:\Users\User\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [39]:
base_model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
)

base_model.trainable = False

add_model = Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dense(2, activation='softmax'))

model = add_model
model.compile(loss='categorical_crossentropy', optimizer = optimizers.SGD(lr=LR, momentum = 0.9), metrics = ['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 4098      
Total params: 21,806,882
Trainable params: 4,098
Non-trainable params: 21,802,784
_________________________________________________________________


In [43]:
kf = KFold(10)

X = np.array([i[0] for i in DATA_SET]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
Y = np.array([i[1] for i in DATA_SET])
fold = 0

for train, test in kf.split(DATA_SET):
    fold += 1
    x_train = X[train]
    y_train = Y[train]
    x_test = X[test]
    y_test = Y[test]

    model.fit(x_train, y_train, validation_data=(x_test, y_test), verbose = 1, epochs= 5, batch_size=64)
    Model_name = 'movie-model-fold' + str(fold) + '.h5'
    weights_name = 'movie-weight-fold' + str(fold) + '.h5'
    model.save(Model_name, overwrite = True, include_optimizer = True)
    model.save_weights(weights_name, overwrite=True)

Epoch 1/5
136/136 [==============================] - 25s 183ms/step - loss: 0.0161 - accuracy: 0.9961 - val_loss: 0.0398 - val_accuracy: 0.9938
Epoch 2/5
136/136 [==============================] - 25s 184ms/step - loss: 0.0160 - accuracy: 0.9977 - val_loss: 0.0150 - val_accuracy: 0.9969
Epoch 3/5
136/136 [==============================] - 25s 185ms/step - loss: 0.0394 - accuracy: 0.9931 - val_loss: 0.0220 - val_accuracy: 0.9959
Epoch 4/5
136/136 [==============================] - 25s 186ms/step - loss: 0.0124 - accuracy: 0.9971 - val_loss: 0.0261 - val_accuracy: 0.9969
Epoch 5/5
136/136 [==============================] - 25s 186ms/step - loss: 0.0104 - accuracy: 0.9971 - val_loss: 0.0098 - val_accuracy: 0.9990
Epoch 1/5
136/136 [==============================] - 25s 185ms/step - loss: 0.0067 - accuracy: 0.9986 - val_loss: 0.0161 - val_accuracy: 0.9990
Epoch 2/5
136/136 [==============================] - 26s 189ms/step - loss: 0.0210 - accuracy: 0.9962 - val_loss: 0.0144 - val_accuracy: